In [7]:
import pathlib

import crowsetta
import numpy as np
import pandas as pd

In [2]:
cd ~/Documents/data/vocal/tweetynet-canary-dataset

/home/ildefonso/Documents/data/vocal/tweetynet-canary-dataset


In [3]:
ls

doi_10.5061_dryad.xgxd254f4__v8.zip  llb16_data_matrices.zip
from-dropbox/                        llb16_data.zip
llb11_data/                          llb3_data/
llb11_data_matrices/                 llb3_data_matrices/
llb11_data_matrices.zip              llb3_data_matrices.zip
llb11_data.zip                       llb3_data.zip
llb16_data/                          README.txt
llb16_data_matrices/


In [5]:
BIRD_IDS = [
    'llb3',
    'llb11',
    'llb16',
]

Fix annotation files to be in current crowsetta format

In [19]:
scribe = crowsetta.Transcriber(format='generic-seq')

root = pathlib.Path('~/Documents/data/vocal/tweetynet-canary-dataset').expanduser()

for bird_id in BIRD_IDS:
    wav_root = root / f'{bird_id}_data/{bird_id}_songs'  # destination for csv annotations
    annot_file = root / f'{bird_id}_data/{bird_id}_annot.csv'
    assert wav_root.exists()
    df = pd.read_csv(annot_file)
    # fix column names
    df = df.rename(columns={"onset_Hz": "onset_sample", "offset_Hz": "offset_sample", "audio_file": "notated_path", "annot_file": "annot_path"})
    df = crowsetta.formats.seq.generic.GenericSeqSchema(df)  # if this errors, we didn't fix names right
    df.to_csv(annot_file, index=False)
    # generic_seq = scribe.from_file(annot_file)
    # for annot in generic_seq.annots:
    #     simple_seq = crowsetta.formats.seq.SimpleSeq(onsets_s=annot.onsets_s, offsets_s=annot.offsets_s, 
    #                                                  labels=annot.labels, annot_path=audio_path)
    #     simple_seq.to_file(annot_path=dst / f'{filenum}.wav.csv')

In [29]:
scribe = crowsetta.Transcriber(format='generic-seq')

root = pathlib.Path('~/Documents/data/vocal/tweetynet-canary-dataset').expanduser()

for bird_id in BIRD_IDS:
    wav_root = root / f'{bird_id}_data/{bird_id}_songs'  # destination for csv annotations
    annot_file = root / f'{bird_id}_data/{bird_id}_annot.csv'
    assert wav_root.exists()
    generic_seq = scribe.from_file(annot_file)
    for annot in generic_seq.annots:
        simple_seq = crowsetta.formats.seq.SimpleSeq(onsets_s=annot.seq.onsets_s, offsets_s=annot.seq.offsets_s, 
                                                     labels=annot.seq.labels, annot_path=annot.notated_path)
        simple_seq.to_file(annot_path=wav_root / f'{annot.notated_path.name}.csv')

In [31]:
import shutil

for bird_id in BIRD_IDS:
    print('sorting: ', bird_id)
    wav_root = root / f'{bird_id}_data/{bird_id}_songs'  # destination for csv annotations
    no_annot_dir = wav_root / 'no_annot'
    no_annot_dir.mkdir(exist_ok=True)
    
    wavs = sorted(wav_root.glob('*.wav'))
    for wav in wavs:
        if not (wav.parent / (wav.name + '.csv')).exists():
            shutil.move(wav, no_annot_dir)

sorting:  llb3
sorting:  llb11
sorting:  llb16
